In [12]:
import pandas as pd

def classify_mineral(row):
    mineral_types = []
    
    if 41 < row['SiO2'] < 49 and 12 < row['MgO'] < 17 and 9 < row['FeO'] < 14 and row['Na2O'] < 3 and 10 < row['CaO'] < 12 and 6 < row['Al2O3'] < 13 and row['TOTAL'] < 100 and row['TiO2'] > 1:
        mineral_types.append('Amph')
    if 51 < row['SiO2'] and 14 < row['MgO'] < 17 and row['FeO'] < 9 and row['Al2O3'] > 1 and row['CaO'] > 21:
        mineral_types.append('Cpx')
    if 52 < row['SiO2'] < 57 and 22 < row['MgO'] < 30 and 10 < row['FeO'] < 19 and row['Al2O3'] < 2 and row['CaO'] < 1:
        mineral_types.append('Opx')
    if row['SiO2'] > 47 and row['Al2O3'] > 20 and row['Na2O'] > 2 and row['CaO'] > 6:
        mineral_types.append('Plagioclase')
    if row['MgO'] > 40:
        mineral_types.append('Olv')
    if 36 < row['SiO2'] < 41 and 7 < row['Al2O3'] < 17 and 16 < row['MgO'] and 7 < row['K2O'] and row['FeO'] < 13:
        mineral_types.append('Bio')
    return ', '.join(mineral_types) if mineral_types else 'Review'

file_path = '/Users/tomassalazar/Desktop/Work/Krawcyzski Group/ESPM.Code/Petrology/Kamchatka/EPMA S.2 Analyses/EPMA RAW DATA.2/Tomas Salazar 7-22-2024 samples.xlsx'
sheet_name = '1717B1'

try:
    data = pd.read_excel(file_path, sheet_name=sheet_name)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    raise

data.columns = data.columns.str.strip()

data['Mineral_Type'] = data.apply(classify_mineral, axis=1)

print(data[['SAMPLE', 'Mineral_Type']])


         SAMPLE Mineral_Type
0   1717B1 L1.1          Bio
1   1717B1 L1.2          Bio
2   1717B1 L1.3          Opx
3   1717B1 L1.4          Opx
4   1717B1 L1.5          Opx
5   1717B1 L1.6          Bio
6   1717B1 L2.1          Bio
7   1717B1 L2.2          Opx
8   1717B1 L2.3          Opx
9   1717B1 L2.4          Bio
10  1717B1 L2.5          Bio
11  1717B1 L3.1          Bio
12  1717B1 L3.2          Bio
13  1717B1 L3.3          Bio
14  1717B1 L3.4          Opx
15  1717B1 L3.5          Bio
16  1717B1 L3.6          Opx


In [1]:
import pandas as pd
import numpy as np
import os
file_path = '/Users/tomassalazar/Desktop/Work/Krawcyzski Group/ESPM.Code/Petrology/Kamchatka/EPMA S.2 Analyses/EPMA RAW DATA.2/Tomas Salazar 7-22-2024 samples.xlsx'
sheet_name = '7550'

oxide_columns = ['Na2O', 'K2O', 'CaO', 'MnO', 'Al2O3', 'FeO', 'MgO', 'TiO2', 'SiO2', 'Cl', 'F', 'O']
site_columns = ['Na', 'K', 'Ca', 'Mn', 'Al', 'Fe', 'Mg', 'Ti', 'Si', 'Cl', 'F', 'O']

molecular_weights = {
    'Na2O': 61.978, 
    'K2O': 94.196,  
    'CaO': 56.077,   
    'MnO': 70.937,   
    'Al2O3': 101.961, 
    'FeO': 71.844,   
    'MgO': 40.304,   
    'TiO2': 79.866,  
    'SiO2': 60.084,  
    'Cl': 35.453,    
    'F': 18.998,     
    'O': 15.999      
}

number_oxygens = np.array([1, 1, 1, 1, 3, 1, 1, 2, 2, 1, 1, 1])  # Na2O, K2O, CaO, MnO, Al2O3, FeO, MgO, TiO2, SiO2, Cl, F, O

try:
    data = pd.read_excel(file_path, sheet_name=sheet_name, usecols=oxide_columns)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    raise

data.columns = data.columns.str.strip()

sample_ids = data.index + 1 
results = []
for index, row in data.iterrows():
    weight_percent_oxides = row[oxide_columns].values
    
    molecular_proportion_oxides = np.array([weight_percent_oxides[i] / molecular_weights[oxide] for i, oxide in enumerate(oxide_columns)])
    
    oxygen_atomic_proportion = molecular_proportion_oxides * number_oxygens
    
    sum_oxy_ato_prop = np.sum(oxygen_atomic_proportion)
    
    normalization_factor = 22 / sum_oxy_ato_prop
    
    number_anions = oxygen_atomic_proportion * normalization_factor
    number_oxygen_per_cation = np.array([1, 1, 1, 1, 3, 1, 1, 2, 2, 1, 1, 1])
    
    cations = number_anions / number_oxygen_per_cation
    results.append(cations)
    
results_df = pd.DataFrame(results, columns=site_columns)
results_df.insert(0, 'Sample ID', sample_ids)
csv_path = os.path.join(os.path.dirname(file_path), '7550 amphibole_formula_numbers_per_sample.csv')
results_df.to_csv(csv_path, index=False)

print(f"CSV file saved to: {csv_path}")


CSV file saved to: /Users/tomassalazar/Desktop/Work/Krawcyzski Group/ESPM.Code/Petrology/Kamchatka/EPMA S.2 Analyses/EPMA RAW DATA.2/7550 amphibole_formula_numbers_per_sample.csv
